# About the dataset
This contains data of news headlines published over a period of 15 years. From the reputable Australian news source ABC (Australian Broadcasting Corp.)
Site: http://www.abc.net.au/
Prepared by Rohit Kulkarni


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

import re
import nltk
import os, glob
import pandas as pd

import chardet

import urduhack
urduhack.download()
from urduhack.normalization import normalize
from urduhack import preprocessing as ppro

import json
from urduhack.config import LEMMA_LOOKUP_TABLE_PATH

In [3]:
# Remove stop words from text
from typing import FrozenSet

# Urdu Language Stop words list
STOP_WORDS: FrozenSet[str] = frozenset("""
 آ آئی آئیں آئے آتا آتی آتے آس آمدید آنا آنسہ آنی آنے آپ آگے آہ آہا آیا اب ابھی ابے
 ارے اس اسکا اسکی اسکے اسی اسے اف افوہ البتہ الف ان اندر انکا انکی انکے انہوں انہی انہیں اوئے اور اوپر
 اوہو اپ اپنا اپنوں اپنی اپنے اپنےآپ اکثر اگر اگرچہ اہاہا ایسا ایسی ایسے ایک بائیں بار بارے بالکل باوجود باہر
 بج بجے بخیر بشرطیکہ بعد بعض بغیر بلکہ بن بنا بناؤ بند بڑی بھر بھریں بھی بہت بہتر تاکہ تاہم تب تجھ
 تجھی تجھے ترا تری تلک تم تمام تمہارا تمہاروں تمہاری تمہارے تمہیں تو تک تھا تھی تھیں تھے تیرا تیری تیرے
 جا جاؤ جائیں جائے جاتا جاتی جاتے جانی جانے جب جبکہ جدھر جس جسے جن جناب جنہوں جنہیں جو جہاں جی جیسا
 جیسوں جیسی جیسے حالانکہ حالاں حصہ حضرت خاطر خالی خواہ خوب خود دائیں درمیان دریں دو دوران دوسرا دوسروں دوسری دوں
 دکھائیں دی دیئے دیا دیتا دیتی دیتے دیر دینا دینی دینے دیکھو دیں دیے دے ذریعے رکھا رکھتا رکھتی رکھتے رکھنا رکھنی
 رکھنے رکھو رکھی رکھے رہ رہا رہتا رہتی رہتے رہنا رہنی رہنے رہو رہی رہیں رہے ساتھ سامنے ساڑھے سب سبھی
 سراسر سمیت سوا سوائے سکا سکتا سکتے سہ سہی سی سے شاید شکریہ صاحب صاحبہ صرف ضرور طرح طرف طور علاوہ عین
 فقط فلاں فی قبل قطا لئے لائی لائے لاتا لاتی لاتے لانا لانی لانے لایا لو لوجی لوگوں لگ لگا لگتا
 لگتی لگی لگیں لگے لہذا لی لیا لیتا لیتی لیتے لیکن لیں لیے لے ماسوا مت مجھ مجھی مجھے محترم محترمہ محض
 مرا مرحبا مری مرے مزید مس مسز مسٹر مطابق مل مکرمی مگر مگھر مہربانی میرا میروں میری میرے میں نا نزدیک
 نما نہ نہیں نیز نیچے نے و وار واسطے واقعی والا والوں والی والے واہ وجہ ورنہ وغیرہ ولے وگرنہ وہ وہاں
 وہی وہیں ویسا ویسے ویں پاس پایا پر پس پلیز پون پونی پونے پھر پہ پہلا پہلی پہلے پیر پیچھے چاہئے
 چاہتے چاہیئے چاہے چلا چلو چلیں چلے چناچہ چند چونکہ چکی چکیں چکے ڈالنا ڈالنی ڈالنے ڈالے کئے کا کاش کب کبھی
 کدھر کر کرتا کرتی کرتے کرم کرنا کرنے کرو کریں کرے کس کسی کسے کم کن کنہیں کو کوئی کون کونسا
 کونسے کچھ کہ کہا کہاں کہہ کہی کہیں کہے کی کیا کیسا کیسے کیونکر کیونکہ کیوں کیے کے گئی گئے گا گنا
 گو گویا گی گیا ہائیں ہائے ہاں ہر ہرچند ہرگز ہم ہمارا ہماری ہمارے ہمی ہمیں ہو ہوئی ہوئیں ہوئے ہوا
 ہوبہو ہوتا ہوتی ہوتیں ہوتے ہونا ہونگے ہونی ہونے ہوں ہی ہیلو ہیں ہے یا یات یعنی یک یہ یہاں یہی یہیں
""".split())


def remove_stopwords(text: str):
    return " ".join(word for word in text.split() if word not in STOP_WORDS)

_WORD2LEMMA = None

def lemma_lookup(text, lookup_path: str = LEMMA_LOOKUP_TABLE_PATH) -> list:
    global _WORD2LEMMA
    if _WORD2LEMMA is None:
        with open(lookup_path, "r", encoding="utf-8") as file:
            _WORD2LEMMA = json.load(file)
            
    tokens = nltk.WordPunctTokenizer().tokenize(text)
    
    return [_WORD2LEMMA[word] if word in _WORD2LEMMA else word for word in tokens]

def preprocessing(text):
    text = normalize(text)
    text = ppro.normalize_whitespace(text)
    text = ppro.remove_punctuation(text)
    text = ppro.remove_accents(text)
    text = ppro.replace_urls(text)
    text = ppro.replace_emails(text)
    text = ppro.replace_phone_numbers(text)
    text = ppro.replace_numbers(text)
    text = ppro.replace_currency_symbols(text)
    text = ppro.remove_english_alphabets(text)    
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'<br />', ' ', text)
    text = remove_stopwords(text)
    text = lemma_lookup(text)

    return text

In [25]:
data_path = "./urdu"
data = pd.DataFrame(columns=["Data"])
for file_path in glob.glob(os.path.join(data_path, '*.txt')):
    with open(file_path, 'rb') as f:
        text = f.read()
        encoding = chardet.detect(text)['encoding']
        text = text.decode(encoding, errors='replace')
        data = data.append({"Data": text}, ignore_index=True)        

C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

...

C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"Data": text}, ignore_index=True)
C:\Users\Muhammad_Haseeb\AppData\Local\Temp\ipykernel_14932\483125042.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

In [26]:
# data = pd.read_csv("../input/abcnews-date-text.csv",error_bad_lines=False,usecols =["headline_text"])
# data.head()

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    500 non-null    object
dtypes: object(1)
memory usage: 4.0+ KB


# Deleting dupliate headlines(if any)

In [28]:
data[data['Data'].duplicated(keep=False)].sort_values('Data').head(8)

,Data
305,"""*مسلم لیگ ن کم از کم مذاق والے پارٹی الیکشن ہ..."
266,"""*مسلم لیگ ن کم از کم مذاق والے پارٹی الیکشن ہ..."
323,"""الیکشن کمیشن نے سابق وزیر اعظم عمران خان، اسد..."
318,"""الیکشن کمیشن نے سابق وزیر اعظم عمران خان، اسد..."
346,"""ایم کیو ایم کے دھڑوں کا متحد ہونے کا اعلان، ’..."
295,"""ایم کیو ایم کے دھڑوں کا متحد ہونے کا اعلان، ’..."
257,"""ایم کیو ایم کے دھڑوں کا متحد ہونے کا اعلان، ’..."
352,"""تحریک انصاف نے اعتماد کے ووٹ کے اجلاس سے غیرح..."


In [29]:
data["Data"] = data["Data"].apply(preprocessing)

In [30]:
desc = [value for value in data["Data"].values]

In [34]:
from sklearn.feature_extraction import text

In [36]:
bow_transform = text.CountVectorizer(tokenizer=lambda doc: doc, lowercase=False) 
tfidf_transform = text.TfidfTransformer()
        
bow = bow_transform.fit_transform(data) 

vectorizer = tfidf_transform.fit_transform(bow)

In [37]:
word_features = vectorizer.get_feature_names()
print(len(word_features))
print(word_features[5000:5100])

AttributeError: get_feature_names not found

In [ ]:
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')

def tokenize(text):
    return [stemmer.stem(word) for word in tokenizer.tokenize(text.lower())]

# Vectorization with stop words(words irrelevant to the model), stemming and tokenizing

In [ ]:
vectorizer2 = TfidfVectorizer(stop_words = stop_words, tokenizer = tokenize)
X2 = vectorizer2.fit_transform(desc)
word_features2 = vectorizer2.get_feature_names()
print(len(word_features2))
print(word_features2[:50]) 

In [ ]:
vectorizer3 = TfidfVectorizer(stop_words = stop_words, tokenizer = tokenize, max_features = 1000)
X3 = vectorizer3.fit_transform(desc)
words = vectorizer3.get_feature_names()

For this, we will use k-means clustering algorithm.
# K-means clustering
(Source [Wikipedia](https://en.wikipedia.org/wiki/K-means_clustering#Standard_algorithm) )
![http://gdurl.com/5BbP](http://gdurl.com/5BbP)"

# Elbow method to select number of clusters
This method looks at the percentage of variance explained as a function of the number of clusters: One should choose a number of clusters so that adding another cluster doesn't give much better modeling of the data. More precisely, if one plots the percentage of variance explained by the clusters against the number of clusters, the first clusters will add much information (explain a lot of variance), but at some point the marginal gain will drop, giving an angle in the graph. The number of clusters is chosen at this point, hence the "elbow criterion". This "elbow" cannot always be unambiguously identified. Percentage of variance explained is the ratio of the between-group variance to the total variance, also known as an F-test. A slight variation of this method plots the curvature of the within group variance.
# Basically, number of clusters = the x-axis value of the point that is the corner of the "elbow"(the plot looks often looks like an elbow)

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=10,random_state=0)
    kmeans.fit(X3)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.savefig('elbow.png')
plt.show()

As more than one elbows have been generated, I will have to select right amount of clusters by trial and error. So, I will showcase the results of different amount of clusters to find out the right amount of clusters.

In [ ]:
print(words[250:300])

# 3 Clusters

In [ ]:
kmeans = KMeans(n_clusters = 3, n_init = 20, n_jobs = 1) # n_init(number of iterations for clsutering) n_jobs(number of cpu cores to use)
kmeans.fit(X3)
# We look at 3 the clusters generated by k-means.
common_words = kmeans.cluster_centers_.argsort()[:,-1:-26:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

# 5 Clusters

In [ ]:
kmeans = KMeans(n_clusters = 5, n_init = 20, n_jobs = 1)
kmeans.fit(X3)
# We look at 5 the clusters generated by k-means.
common_words = kmeans.cluster_centers_.argsort()[:,-1:-26:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

# 6 Clusters

In [ ]:
kmeans = KMeans(n_clusters = 6, n_init = 20, n_jobs = 1)
kmeans.fit(X3)
# We look at 6 the clusters generated by k-means.
common_words = kmeans.cluster_centers_.argsort()[:,-1:-26:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

# 8 Clusters

In [ ]:
kmeans = KMeans(n_clusters = 8, n_init = 20, n_jobs = 1)
kmeans.fit(X3)
# Finally, we look at 8 the clusters generated by k-means.
common_words = kmeans.cluster_centers_.argsort()[:,-1:-26:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

Because even I didn't know what kind of clusters would be generated, I will describe them in comments.